In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/protonx-cloud-storage/data.txt
data = open('data.txt').read()

--2020-08-23 12:29:21--  https://storage.googleapis.com/protonx-cloud-storage/data.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.200.128, 74.125.68.128, 172.217.194.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.200.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 93578 (91K) [text/plain]
Saving to: ‘data.txt’

data.txt            100%[===================>]  91.38K  --.-KB/s    in 0.001s  

2020-08-23 12:29:22 (134 MB/s) - ‘data.txt’ saved [93578/93578]



In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
import tensorflow.keras.utils as ku
import numpy as np

In [ ]:
corpus = data.lower().split("\n")

In [ ]:
corpus

['from fairest creatures we desire increase,',
 "that thereby beauty's rose might never die,",
 'but as the riper should by time decease,',
 'his tender heir might bear his memory:',
 'but thou, contracted to thine own bright eyes,',
 "feed'st thy light'st flame with self-substantial fuel,",
 'making a famine where abundance lies,',
 'thyself thy foe, to thy sweet self too cruel.',
 "thou that art now the world's fresh ornament",
 'and only herald to the gaudy spring,',
 'within thine own bud buriest thy content',
 'and, tender churl, makest waste in niggarding.',
 'pity the world, or else this glutton be,',
 "to eat the world's due, by the grave and thee.",
 'when forty winters shall beseige thy brow,',
 "and dig deep trenches in thy beauty's field,",
 "thy youth's proud livery, so gazed on now,",
 "will be a tatter'd weed, of small worth held:",
 "then being ask'd where all thy beauty lies,",
 'where all the treasure of thy lusty days,',
 'to say, within thine own deep-sunken eyes,',

## Yêu cầu: Sinh từ tiếp khi gõ bất cứ một câu nào có chiều dài bất kỳ.

In [ ]:
# Dự đoán 10 từ tiếp theo
# despite of wrinkles -> this thy golden time to heart's sight ' must '
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

# create input sequences using list of tokens
input_sequences = []
for line in corpus:
  token_list = tokenizer.texts_to_sequences([line])[0]
  for i in range(1, len(token_list)):
    n_gram_sequence = token_list[:i+1]
    print(n_gram_sequence)
    input_sequences.append(n_gram_sequence)

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
[153, 5, 174, 3]
[153, 5, 174, 3, 856]
[153, 5, 174, 3, 856, 857]
[66, 209]
[66, 209, 2595]
[66, 209, 2595, 287]
[66, 209, 2595, 287, 150]
[66, 209, 2595, 287, 150, 2596]
[858, 206]
[858, 206, 1]
[858, 206, 1, 1299]
[858, 206, 1, 1299, 13]
[858, 206, 1, 1299, 13, 23]
[858, 206, 1, 1299, 13, 23, 5]
[858, 206, 1, 1299, 13, 23, 5, 400]
[858, 206]
[858, 206, 1]
[858, 206, 1, 1299]
[858, 206, 1, 1299, 2597]
[858, 206, 1, 1299, 2597, 3]
[858, 206, 1, 1299, 2597, 3, 160]
[858, 206, 1, 1299, 2597, 3, 160, 249]
[1, 7]
[1, 7, 29]
[1, 7, 29, 205]
[1, 7, 29, 205, 13]
[1, 7, 29, 205, 13, 5]
[1, 7, 29, 205, 13, 5, 488]
[1, 7, 29, 205, 13, 5, 488, 411]
[348, 2598]
[348, 2598, 7]
[348, 2598, 7, 66]
[348, 2598, 7, 66, 26]
[348, 2598, 7, 66, 26, 1298]
[348, 2598, 7, 66, 26, 1298, 471]
[348, 2598, 7, 66, 26, 1298, 471, 1258]
[858, 206]
[858, 206, 1]
[858, 206, 1, 75]
[858, 206, 1, 75, 147]
[858, 206, 1, 75, 147, 39]
[858, 206, 1, 75, 147, 39, 985]


In [ ]:
tokenizer.word_index

{'and': 1,
 'the': 2,
 'to': 3,
 'of': 4,
 'my': 5,
 'i': 6,
 'in': 7,
 'that': 8,
 'thy': 9,
 'thou': 10,
 'with': 11,
 'for': 12,
 'is': 13,
 'love': 14,
 'not': 15,
 'but': 16,
 'a': 17,
 'me': 18,
 'thee': 19,
 'so': 20,
 'be': 21,
 'as': 22,
 'all': 23,
 'you': 24,
 'his': 25,
 'which': 26,
 'when': 27,
 'it': 28,
 'this': 29,
 'by': 30,
 'your': 31,
 'doth': 32,
 'do': 33,
 'from': 34,
 'on': 35,
 'or': 36,
 'no': 37,
 'then': 38,
 'have': 39,
 'what': 40,
 'are': 41,
 'if': 42,
 'more': 43,
 'mine': 44,
 'their': 45,
 'shall': 46,
 'sweet': 47,
 'time': 48,
 'will': 49,
 'they': 50,
 'beauty': 51,
 'nor': 52,
 'eyes': 53,
 'art': 54,
 'her': 55,
 'heart': 56,
 'yet': 57,
 'o': 58,
 'than': 59,
 'can': 60,
 'should': 61,
 'thine': 62,
 'now': 63,
 'where': 64,
 'make': 65,
 'one': 66,
 'hath': 67,
 'he': 68,
 'fair': 69,
 'still': 70,
 'how': 71,
 'eye': 72,
 'him': 73,
 'like': 74,
 'true': 75,
 'see': 76,
 'am': 77,
 'she': 78,
 'those': 79,
 'though': 80,
 'being': 81,
 'some'

### 1. Xử lý dữ liệu. Chia features, label

In [ ]:
# pad sequences
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# create predictors and label
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
print(label)
label = ku.to_categorical(label, num_classes=total_words)

[ 417  877  166 ... 3210   15   14]


In [ ]:
predictors

array([[   0,    0,    0, ...,    0,    0,   34],
       [   0,    0,    0, ...,    0,   34,  417],
       [   0,    0,    0, ...,   34,  417,  877],
       ...,
       [   0,    0,    0, ..., 3209,  493,  493],
       [   0,    0,    0, ...,  493,  493, 3210],
       [   0,    0,    0, ...,  493, 3210,   15]], dtype=int32)

### 2. Xây dựng model
#### Yêu cầu độ chính xác: 80%

In [ ]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(150, return_sequences = True)))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words/2, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 10, 100)           321100    
_________________________________________________________________
bidirectional (Bidirectional (None, 10, 300)           301200    
_________________________________________________________________
dropout (Dropout)            (None, 10, 300)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               160400    
_________________________________________________________________
dense (Dense)                (None, 1605)              162105    
_________________________________________________________________
dense_1 (Dense)              (None, 3211)              5156866   
Total params: 6,101,671
Trainable params: 6,101,671
Non-trainable params: 0
______________________________________________

In [ ]:
 history = model.fit(predictors, label, epochs=100, verbose=1)

### 3. Dự đoán 10 từ tiếp theo

In [ ]:
test_seq = 'despite of wrinkles'
# 'despite of wrinkles this thy' -> 'golden'
# sinh ra 10 từ tiếp theo sau test_seq
# despite of wrinkles this thy golden time to heart's sight ' must '
next_words = 10

for _ in range(next_words):
  token_list = tokenizer.texts_to_sequences([test_seq])[0]
  token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
  predicted = model.predict_classes(token_list, verbose=0)
  output_word = ""
  for word, index in tokenizer.word_index.items():
    if index == predicted:
      output_word = word
      break
  test_seq += " " + output_word
print(test_seq)


Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
despite of wrinkles this thy golden time i can morrow grow leaves words
